# \[Week5] 추천시스템  
## [캐글 데이터](https://www.kaggle.com/tmdb/tmdb-movie-metadata?select=tmdb_5000_movies.csv) tmdb_5000_movies.csv   
참고 사이트 : [[구름 Edu]](https://edu.goorm.io/learn/lecture/20671/%EB%94%A5%EB%9F%AC%EB%8B%9D-%EC%9D%B4%EB%A1%A0-%EB%B0%8F-%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EC%8B%A4%EC%8A%B5)

## 1. 자연어처리 - 문서 유사도 (용어 정리)

1. Bag of Words   

단어들의 순서는 전혀 고려하지 않고, 단어들의 출현 빈도(freqeuncy)에만 집중하는 텍스트 데이터의 수치화 표현 방법이다.

1) 문장 간의 유사도 구하는 방법 

BOW로 각 문장을 binary vector로 표현한 후 각 인덱스의 값을 곱하고 전부 더한다.


e.g.

awesome thank you [1, 1, 1, 0, 0, 0, 0] <br>
great thank you &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[0, 1, 1, 1, 0, 0, 0]

1x1 + 1x1 = 2

2) 단점

- Sparsity

If we use all English words for bag of words, the vector will be very long, but very few non zeros

실제 사전에는 100만개가 넘는 단어들이 있을 수도 있다. 그렇게 되면 벡터의 차원이 100만개가 넘어가기 때문에 실제 문장하나를 표현할 때 대부분의 값이 0이고 그외의 값들은 상당히 적을 것이다. 결국 학습량이 많아지고 컴퓨터 자원도 상당히 많이 사용하게 된다.

e.g. the game is fun [1,1,1,1,0,0,0,0,0,0,0,,,,,,0,0,0,0,0]

- Frequent words has more power

많이 출현한 단어는 힘이 세진다. 만약 의미없는 단어들이 많이 사용 되었다면 우리가 원하는 결과를 얻기는 어려울 것이다.

- Ignoring word orders

bag of words will have same vector for totally different sentence if they have same words since it is ignoring word orders

e.g. home run VS run home

Cf. n-gram으로 보완 가능

- Out of vocabulary

What if text has mis-spelling? (good moning, tank yo)

What if text is unseen from your dataset? (gr8, gooood)

Bag of words cannot handle unseen words

2. n-그램

contiguous sequence of N tokens (words, characters... )

**(예제) fine thank you**  
1) 1-gram (unigram)

    1-1) Word level  

    [fine,thank,you]

    1-2) Character level

    [f,i,n,e, ,t,h,a,n,k, ,y,o,u]

2) 2-gram (bigram)

    2-1) Word level

    [fine thank,thank you]  

    2-2) Character level 

    [fi,in,ne,e , t,th,ha,an,nk,k , y,yo,ou]

3) 3-gram (trigram)

    3-1) Word level 

    [fine thank you]

    3-2) Character level 

    [fin,ine,ne ,e t, th,tha,han,ank,nk ,k y, yo,you]

4) n-gram 사용하는 이유?  

- Overcome bag of words drawbacks (ignoring sequence of words)

- Next word prediction

- Find Misspelling

- And more...

3. 점 A(3,4), 점 B(4,3)의 cosine 유사도를 구하시오.  

{(3 * 4) + (4 * 3)} / (5 * 5) = 0.96

## 2. 추천시스템  

### 0. 추천시스템 종류 정리

1. Content-based filtering  

아이템이나 유저를 분석하여 비슷한 아이템을 추천한다. 아이템과 유저간의 액션을 분석하는 것이 아니라 컨텐츠 자체를 분석하기 때문에 많은 양의 유저 액션을 요구하지 않는다는 장점이 있다. CF의 단점인 cold start가 없다.

2. Collaborative filtering  

유저의 평가 내역을 이용한다. 아이템이나 유저의 유사도를 모델링하고 측정하여 추천한다.

단점)
- Cold start<br>
    : 유저의 평가 내역이 필요하다. 데이터가 없는 서비스 초기에는 추천이 부정확하다.
- Scalability
- Sparsity

    - user-based collaborative filtering  

    유저의 행위를 측정 및 분석하고 이를 기반으로 유저 간 유사도를 측정한다. 유사도가 높은 유저의 선호 아이템들을 기반으로 추천한다.

    - item-based collaborative filtering  

    아이템 간의 유사도를 측정하며, 유저가 어떤 아이템을 선호하면 유사한 다른 아이템을 추천한다.

Content-based filtering을 해봅시다.  
보통은 장르를 기준으로 많이 하지만, 자연어처리에 좀 더 익숙해지기 위해서 줄거리를 바탕으로 해보겠습니다.  
Collaborative filtering도 한번 직접 해보세요. 재밌습니다:)

### 1. CSV 파일 불러와 DataFrame으로 저장

In [26]:
import pandas as pd
movies = pd.read_csv('tmdb_5000_movies.csv')           # tmdb_5000_movies.csv dataframe으로 읽어오기
movies = movies[["original_title", "overview"]]
movies["overview"] = movies["overview"].astype("str")
movies

,original_title,overview
0,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,Spectre,A cryptic message from Bond’s past sends him o...
3,The Dark Knight Rises,Following the death of District Attorney Harve...
4,John Carter,"John Carter is a war-weary, former military ca..."
...,...,...
4798,El Mariachi,El Mariachi just wants to play his guitar and ...
4799,Newlyweds,A newlywed couple's honeymoon is upended by th...
4800,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic..."
4801,Shanghai Calling,When ambitious New York attorney Sam is sent t...


### 2. 전처리  
출력 예시)

    original_title    0
    overview          3
    dtype: int64


In [27]:
# null 값 개수 확인
movies.isnull().sum()

original_title    0
overview          0
dtype: int64

1) 줄거리가 NaN인 **영화** drop

In [28]:
movies[movies['overview'].isnull()]

,original_title,overview


2) "overview" column 모두 소문자로, 문자+숫자(\w)만 남기고 나머지는 띄어쓰기로 대체

In [29]:
import re
# 되도록 apply, lambda 사용할것!
movies['overview'] = movies['overview'].apply(lambda x: re.sub('[^a-z0-9]+', ' ', x.lower()))

### 3. TF-IDF 계산

1) TF 란?  

Term Frequency

TF measures how frequently a term occurs in a document

If a term occurs more times than other terms in a document, the term has more relevance than other terms for the document

2) DF 란?  

Document Frequency

단어 자체가 문서군 내에서 자주 사용되는 경우, 이것은 그 단어가 흔하게 등장한다는 것을 의미한다.

3) IDF 란?   

Inverse Document Frequency

Log (Total # of Docs / # of Docs with the term in it) <br>
Cf. Docs: 문장, term: 단어

TF-IDF: To find how relevant a term in a document (numerically) <br>
relevant: a word relevance is the amount of info that gives about its context

라이브러리로 적용해보자...

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
# ngram_range=(1, 2) 는 단어를 1개 혹은 2개 연속으로 보겠다
tfidf_vec = TfidfVectorizer(ngram_range = (1,2))        # Vectorizer 생성
tfidf_matrix = tfidf_vec.fit_transform(movies['overview'])       # Vectorizer가 단어들을 학습

### 4. 영화 간 **cosine 유사도**  
\[출력예시]  

    ### COSINE Similarity ###
    [[1.         0.01513636 0.00614044 ... 0.01195144 0.00571986 0.00630023]
    [0.01513636 1.         0.01307772 ... 0.01768395 0.00997186 0.00666392]
    [0.00614044 0.01307772 1.         ... 0.01288316 0.00565125 0.00612543]
    ...
    [0.01195144 0.01768395 0.01288316 ... 1.         0.01532266 0.0089986 ]
    [0.00571986 0.00997186 0.00565125 ... 0.01532266 1.         0.01649368]
    [0.00630023 0.00666392 0.00612543 ... 0.0089986  0.01649368 1.        ]]
    [[   0 3603  634 ... 2852 3945 3386]
    ### 유사도 기준 index 정렬 ###
    [[   0 3603  634 ... 2852 3945 3386]
    [   1 2379 2542 ... 3487 3622 3945]
    [   2 1343 3161 ... 3534 3637 3082]
    ...
    [4797 4033  569 ... 2604 3685 1658]
    [4798 2017 1480 ... 2108 1658 4511]
    [4799 2586  868 ... 3987 2192  669]]



In [31]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
plot_similarity = cosine_similarity(tfidf_matrix, tfidf_matrix)        # 줄거리 간 cosine 유사도 구하기 - tfidf_matrix 사용
print("### COSINE Similarity ###")
print(plot_similarity)
similar_index = np.argsort(-plot_similarity)            # 유사도 높은 순서대로 index 정렬
print("### 유사도 기준 index 정렬 ###") 
print(similar_index)

### COSINE Similarity ###
[[1.         0.01514414 0.00614504 ... 0.01195829 0.00572386 0.006304  ]
 [0.01514414 1.         0.01308528 ... 0.01769221 0.00997909 0.00666831]
 [0.00614504 0.01308528 1.         ... 0.01289    0.00565554 0.00612954]
 ...
 [0.01195829 0.01769221 0.01289    ... 1.         0.01532978 0.00900306]
 [0.00572386 0.00997909 0.00565554 ... 0.01532978 1.         0.01649947]
 [0.006304   0.00666831 0.00612954 ... 0.00900306 0.01649947 1.        ]]
### 유사도 기준 index 정렬 ###
[[   0 3604  634 ... 4140 2596 2669]
 [   1 2379 2542 ...  161 2656 4458]
 [   2 1343 3162 ... 4144  159  559]
 ...
 [4800 4034  569 ... 2656 4431 1038]
 [4801 2017 1480 ... 3784 4458 2108]
 [4802 2586  868 ...  669 1273 3686]]


In [32]:
input_movie = "Avatar"          # data에 있는 영화의 제목을 넣어야 합니다.

movie_index = movies[movies['original_title']=='Avatar'].index[0]    # input_movie에 해당하는 index 값 가져오기
similar_movies = similar_index[movie_index][:10]        # 유사도 상위 10개 index 가져오기
# 인덱스로 사용하기 위해서는 1차원으로 변형 (Q. 이미 1차원!!?!)
#similar_movies_index = similar_movies.reshape(-1)             # similar_movies 1차원 변형
display(movies.iloc[similar_movies])

,original_title,overview
0,Avatar,in the 22nd century a paraplegic marine is dis...
3604,Apollo 18,officially apollo 17 was the last manned missi...
634,The Matrix,set in the 22nd century the matrix tells the s...
529,Tears of the Sun,navy seal lieutenant a k waters and his elite ...
1013,Child 44,set in stalin era soviet union a disgraced mgb...
775,Supernova,set in the 22nd century when a battered salvag...
2130,The American,dispatched to a small italian town to await fu...
2628,Blood and Chocolate,a young teenage werewolf is torn between honor...
1341,Obitaemyy Ostrov,on the threshold of 22nd century furrowing the...
2808,The Exorcist,12 year old regan macneil begins to adapt an e...


3주동안의 자연어처리 스터디가 끝났습니다. 다음주부터는 이미지처리 스터디가 시작됩니다.  
짧은 기간이라 많은 부분을 다루지는 못했지만, 추후에 프로젝트를 진행하면서 멤버들끼리 딥러닝 등 심화 부분을 공부하시면 더 좋을 것 같습니다.  
중간에 시험기간도 겹쳐서 바쁜 와중에도 과제 잘 해주셔서 감사합니다!

\[답변] 정말 수고 많으셨습니다!!!! 감사합니다💙